# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,biltine,14.5333,20.9167,27.70,20,92,3.64,TD,1729727536
1,1,sitka,57.0531,-135.3300,6.94,76,75,0.00,US,1729727538
2,2,marquette,46.5333,-87.6335,4.09,81,46,0.45,US,1729727539
3,3,yellowknife,62.4560,-114.3525,1.29,99,20,1.54,CA,1729727540
4,4,weno,7.4515,151.8468,29.97,84,100,4.63,FM,1729727542
...,...,...,...,...,...,...,...,...,...,...
542,542,takoradi,4.8845,-1.7554,24.90,90,60,3.09,GH,1729728169
543,543,taro,-8.3862,115.2800,23.43,78,61,0.86,ID,1729728170
544,544,mahibadhoo,3.7833,72.9667,27.94,78,66,7.03,MV,1729728172
545,545,chernomorskoye,45.5039,32.6997,15.34,68,31,6.49,UA,1729728173


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [70]:


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points (
    "Lng",
    "Lat",
    geo = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600
)
# Display the map
map_plot_1

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition_df= city_data_df[city_data_df["Cloudiness"]==0]

# Drop any rows with null values
ideal_condition_df = ideal_condition_df.dropna()

# Display sample data
print(ideal_condition_df)

     City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
9          9       ushuaia -54.8000  -68.3000      8.81        61           0   
26        26       stanley  54.8680   -1.6985     10.01        89           0   
31        31  punta arenas -53.1500  -70.9167     10.06        62           0   
44        44      la passe  45.5549   -0.8967     13.82        90           0   
45        45         pisco -13.7000  -76.2167     19.03        82           0   
..       ...           ...      ...       ...       ...       ...         ...   
511      511        dakhla  23.6848  -15.9580     20.30        88           0   
517      517       salinas  36.6777 -121.6555     22.10        61           0   
521      521      rockport  28.0206  -97.0544     28.02        78           0   
537      537    neukirchen  50.8667    9.3333      4.64        93           0   
541      541   zhangjiakou  40.8100  114.8794      7.71        54           0   

     Wind Speed Country    

### Step 3: Create a new DataFrame called `hotel_df`.

In [56]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition_df [['City', 'Country', 'Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] = " "

# Display sample data
print(hotel_df)

             City Country      Lat       Lng  Humidity Hotel Name
9         ushuaia      AR -54.8000  -68.3000        61           
26        stanley      GB  54.8680   -1.6985        89           
31   punta arenas      CL -53.1500  -70.9167        62           
44       la passe      FR  45.5549   -0.8967        90           
45          pisco      PE -13.7000  -76.2167        82           
..            ...     ...      ...       ...       ...        ...
511        dakhla      EH  23.6848  -15.9580        88           
517       salinas      US  36.6777 -121.6555        61           
521      rockport      US  28.0206  -97.0544        78           
537    neukirchen      DE  50.8667    9.3333        93           
541   zhangjiakou      CN  40.8100  114.8794        54           

[88 rows x 6 columns]


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [68]:
# Set parameters to search for a hotel
limit = 20
radius = 10000



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row ['Lat']
    longitude = row ['Lng']
  

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng}, {Lat}, {radius}"
    params["bias"] =  f"proximity: {Lng}, {Lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    response = requests.get(base_url,params= params)
  

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


NameError: name 'Lng' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)